In [ ]:
# -------------------------------------------------
# 1. System packages (run once)
# -------------------------------------------------
!apt-get -qq update && apt-get -qq install -y ffmpeg   # update apt index (quiet) and install ffmpeg (required for audio I/O)

# -------------------------------------------------
# 2. Install Python dependencies (run once)
# -------------------------------------------------
# install/upgrade the required Python libs (quiet)
# 🤗 Transformers – model & generation utilities
# 🤗 Datasets – optional, not used directly here
# 🤗 Accelerate – optional, for multi‑GPU handling
# 🤗 Hub client – needed for snapshot_download
# Optimum + ONNX Runtime integration
# audio I/O, resampling, progress bars, etc.
!pip install -q --upgrade \
    "transformers" \
    "datasets" \
    "accelerate" \
    "huggingface_hub" \
    "optimum[onnxruntime]" \
    "soundfile" \
    "librosa" \
    "ffmpeg-python" \
    "tqdm"
# -------------------------------------------------
# 3. Install the correct ONNX Runtime build
# -------------------------------------------------
import torch, subprocess, sys, pathlib, shutil   # import utilities we’ll need later

def pip_install(pkgs):                              # tiny helper that runs pip install in a subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q"] + pkgs)

if torch.cuda.is_available():                       # if a GPU is present, install the GPU‑enabled runtime
    pip_install(["onnxruntime-gpu"])
else:                                               # otherwise fall back to the CPU‑only runtime
    pip_install(["onnxruntime"])

# -------------------------------------------------
# 4. Download Whisper‑tiny ONNX repo
# -------------------------------------------------
from huggingface_hub import snapshot_download      # utility to download a repo (or part of it) from HF Hub

onnx_repo_id = "onnx-community/whisper-large-v3-turbo-ONNX"      # repo that contains the ONNX‑exported Whisper‑tiny
onnx_dir = snapshot_download(                      # download only the files we actually need
    repo_id=onnx_repo_id,
    allow_patterns=[                               # keep only the listed patterns
        "*.onnx",                                 # all ONNX model files
        "config.json",
        "*_data",                                 # external weight files (e.g. encoder_model.onnx_data)
        "generation_config.json",                 # model config (used by WhisperProcessor)
        "preprocessor_config.json",               # feature‑extractor config
        "tokenizer_config.json",                  # tokenizer config
        "vocab.json", "merges.txt",              # BPE vocab files
        "added_tokens.json", "special_tokens_map.json",
        "normalizer.json", "tokenizer.json",      # full tokenizer definition
    ],
    local_dir="./whisper-large-v3-turbo-ONNX",               # where to store the files locally
    cache_dir="./hf_cache",                        # shared cache folder (speeds up re‑runs)
)
print("✅ ONNX model downloaded to:", onnx_dir)    # sanity‑check output

# -------------------------------------------------
# 5. Load processor & ONNX model
# -------------------------------------------------
from transformers import WhisperProcessor, GenerationConfig, pipeline   # core HF classes
from optimum.onnxruntime import ORTModelForSpeechSeq2Seq                # ONNX‑accelerated Whisper model

processor = WhisperProcessor.from_pretrained(onnx_dir)   # loads tokenizer + feature extractor from the ONNX folder

execution_provider = (                                 # tell ONNX Runtime where to run the model
    "CUDAExecutionProvider" if torch.cuda.is_available() else "CPUExecutionProvider"
)

ort_model = ORTModelForSpeechSeq2Seq.from_pretrained(  # load the ONNX weights as an HF‑compatible model
    onnx_dir,
    provider=execution_provider,
)

# -------------------------------------------------
# 6. Build a Whisper‑aware GenerationConfig (now works because the JSON exists)
# -------------------------------------------------
gen_cfg = GenerationConfig.from_pretrained(            # reads generation_config.json we just copied
    onnx_dir,                                          # folder containing the JSON
    max_new_tokens=256,                                # limit the length of the generated transcript
    do_sample=False,                                   # Whisper is deterministic → no sampling
    language="de",                                     # default language (German); can be overridden later
    task="transcribe",                                 # default task (transcribe vs. translate)
)

# (Optional sanity‑check – the attribute that caused the earlier crash must exist)
assert hasattr(gen_cfg, "lang_to_id"), "lang_to_id missing – something went wrong!"   # Whisper needs this mapping

ort_model.generation_config = gen_cfg   # attach the config to the ONNX model instance

# -------------------------------------------------
# 7. Build the ASR pipeline
# -------------------------------------------------
asr_pipe = pipeline(                                 # high‑level HF pipeline for automatic‑speech‑recognition
    task="automatic-speech-recognition",
    model=ort_model,                                 # the ONNX Whisper model
    tokenizer=processor.tokenizer,                    # tokenizer from the WhisperProcessor
    feature_extractor=processor.feature_extractor,    # feature extractor (log‑mel spectrogram)
    device=0 if torch.cuda.is_available() else -1,   # 0 → first GPU, -1 → CPU
    # ignore_warning=True   # uncomment to silence the experimental chunk_length_s warning
)

print("✅ Pipeline ready – max_new_tokens:",
      asr_pipe.generation_config.max_new_tokens)   # confirm that the pipeline sees our config

# -------------------------------------------------
# 8. Helper to load audio (wav, mp3, m4a, …)
# -------------------------------------------------
import librosa, numpy as np, soundfile as sf, io, requests   # audio I/O & resampling libs
import textwrap                                   # Standard‑library import for handling long strings

def load_audio(path_or_url, target_sr=16000):
    """Load a local file or a remote URL, convert to mono, and resample to 16 kHz."""
    if isinstance(path_or_url, str) and path_or_url.startswith("http"):   # remote URL?
        resp = requests.get(path_or_url)          # download the file
        resp.raise_for_status()                   # raise if HTTP error
        data, sr = sf.read(io.BytesIO(resp.content))   # read with soundfile from memory
    else:                                          # local path
        data, sr = sf.read(path_or_url)            # read wav/mp3/etc.

    # make mono – Whisper expects a single channel
    if data.ndim > 1:
        data = data.mean(axis=1)                  # average channels

    # resample to the 16 kHz rate Whisper was trained on
    if sr != target_sr:
        data = librosa.resample(data, orig_sr=sr, target_sr=target_sr)

    return data.astype(np.float32)                # ensure float32 (required by the model)

# -------------------------------------------------
# 9. Run inference
# -------------------------------------------------
# Upload a file to /content/audio.wav or give a public URL
audio_path = "/content/audio.wav"          # <-- replace with your own file (or a URL)
audio_array = load_audio(audio_path)       # load & preprocess the audio

audio_input = {"array": audio_array, "sampling_rate": 16000}   # format expected by the pipeline

# NOTE: temperature, top_p, repetition_penalty are ignored for Whisper.
result = asr_pipe(
    audio_input,
    chunk_length_s=30,               # split long audio into 30‑second chunks (optional)
    return_timestamps=False,         # we only want the plain transcript
    generate_kwargs=dict(
        task="transcribe",           # Whisper task – can also be "translate"
        language="de",               # language code (e.g. "en", "de", "fr", …)
        # temperature=0.6,          # ignored by Whisper – left here for illustration
    ),
)


# -------------------------------------------------
# 10. Pretty‑print the transcription
# -------------------------------------------------
raw_text = result["text"]                     # <-- the one‑line string
wrapped   = textwrap.fill(raw_text, width=80) # 80‑char line length (adjust as you like)

print("\n=== Transcription ===")
print(wrapped)